In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
df = pd.read_csv(r'.\booklist.csv')

col_names=['bookId','title','series','author','rating','description','lang','isbn','genres','characters','bookForm','edition','pages','publisher','publishDate','firstPublished','awards','numRating','ratingsByStars','likedPercent','setting','coverImg','bbeScore','bbeVotes','price']
wanted_cols = ['bookId','title','series','author','rating','description','genres','characters','setting','coverImg']

#TODO: filter out our wanted cols, create our index and save it to a file(?)
# filtered_df = df[wanted_cols].head(5)

In [3]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'].values.astype('U'))
# test_df = pd.DataFrame(tfidf_matrix.toarray(), columns = tf.get_feature_names())
# print(test_df)


In [4]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# print(cosine_sim[0])
# print(cosine_sim[1])
# print(cosine_sim[0].shape)


[1.00000000e+00 9.37025755e-04 5.58931176e-03 ... 1.16331301e-02
 0.00000000e+00 6.39679240e-03]
[9.37025755e-04 1.00000000e+00 0.00000000e+00 ... 4.08465597e-03
 1.41892175e-03 0.00000000e+00]
(52478,)


In [8]:
indices = pd.Series(df.index ,index=df['title'])
print(indices.head(10))
idx = indices["Harry Potter and the Order of the Phoenix"]
# sim_scores = list(enumerate(cosine_sim[idx]))


title
The Hunger Games                                                         0
Harry Potter and the Order of the Phoenix                                1
To Kill a Mockingbird                                                    2
Pride and Prejudice                                                      3
Twilight                                                                 4
The Book Thief                                                           5
Animal Farm                                                              6
The Chronicles of Narnia                                                 7
J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings    8
Gone with the Wind                                                       9
dtype: int64


In [6]:
def get_recommendations(title, no_of_recommendation):
    idx = indices[title]
    df['similarity score'] = np.array(cosine_sim[idx])
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # df['similarity score'] = np.array(sim_scores)
    sim_scores = sim_scores[1:no_of_recommendation+1]
    print(len(sim_scores))
    book_indices = [i[0] for i in sim_scores]
    return df[["title", "similarity score"]].iloc[book_indices]

In [7]:
get_recommendations("Harry Potter and the Order of the Phoenix", 10)

10


,title,similarity score
93,Harry Potter and the Prisoner of Azkaban,0.135389
126,Harry Potter and the Chamber of Secrets,0.125192
32,Harry Potter and the Sorcerer's Stone,0.087595
11030,Harry Potter Schoolbooks Box Set: Two Classic ...,0.083425
7008,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",0.067900
103,Harry Potter and the Goblet of Fire,0.066784
51722,Hogwarts: An Incomplete and Unreliable Guide,0.066527
21783,Pronto,0.061132
1600,The Harry Potter Collection 1-4,0.060229
34781,The Unofficial Harry Potter Cookbook: From Cau...,0.058944
